This notebook uses `scholarly` API (https://github.com/scholarly-python-package/scholarly) to enrich CS faculty info

In [1]:
from scrap_cs_faculty import *

In [2]:
org_list = ["CMU-CS","Cornell-CS","MIT-AID","MIT-CS","Stanford-CS","UCB-CS","UIUC-CS",]
idx_org = org_list.index("MIT-CS")
org = org_list[idx_org]
org2 = org.split("-")[0] 

## read CS faculty info scraped previously

In [3]:
file_xlsx = f"faculty-{org}.xlsx"
xlsxf = pd.ExcelFile(file_xlsx)

In [4]:
xlsxf.sheet_names

['Faculty', 'Research Groups']

In [5]:
df = xlsxf.parse('Faculty')

In [6]:
df

,name,job_title,phd_univ,phd_year,research_area,research_concentration,research_focus,url,img_url,phone,email,cell_phone,office_address,department,school
0,Hal Abelson,Class of 1992 Professor,NaN,NaN,"AI and Society, Artificial Intelligence + Deci...",NaN,NaN,https://www.eecs.mit.edu/people/hal-abelson/,https://www.eecs.mit.edu/wp-content/uploads/20...,(617) 253-5856,hal@mit.edu,NaN,Office: 32-G516,Computer Science; AI & Decision-making,Massachusetts Institute Technology
1,Anant Agarwal,"CEO, edX; Professor of EECS;",NaN,NaN,"Computer Architecture, Multicore Processors & ...",NaN,NaN,https://www.eecs.mit.edu/people/anant-agarwal/,https://www.eecs.mit.edu/people/anant-agarwal/,(617) 253-1448,agarwal@mit.edu,NaN,Office: NE55-900,Computer Science; Electrical Engineering,Massachusetts Institute Technology
2,Pulkit Agrawal,Steven and Renee Finn Career Development Profe...,NaN,NaN,"Artificial Intelligence + Machine Learning, Gr...",NaN,NaN,https://www.eecs.mit.edu/people/pulkit-agrawal/,https://www.eecs.mit.edu/people/pulkit-agrawal/,(617) 253-5851,pulkitag@mit.edu,NaN,Office: 32-342,AI & Decision-making; Computer Science,Massachusetts Institute Technology
3,Mohammad Alizadeh,Associate Professor,NaN,NaN,"Multicore Processors & Cloud Computing, Securi...",NaN,NaN,https://www.eecs.mit.edu/people/mohammad-aliza...,https://www.eecs.mit.edu/people/mohammad-aliza...,(617) 253-6042,alizadeh@mit.edu,NaN,Office: 32-G920,Computer Science;,Massachusetts Institute Technology
4,Saman Amarasinghe,Professor of CS and Engineering,NaN,NaN,"Artificial Intelligence + Machine Learning, Co...",NaN,NaN,https://www.eecs.mit.edu/people/saman-amarasin...,https://www.eecs.mit.edu/people/saman-amarasin...,(617) 253-8879,saman@csail.mit.edu,NaN,Office: 38-427,Computer Science,Massachusetts Institute Technology
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,Ryan Williams,Professor of EECS,NaN,NaN,Theory of Computation,NaN,NaN,https://www.eecs.mit.edu/people/ryan-williams/,https://www.eecs.mit.edu/people/ryan-williams/,617-253-5851,rrw@mit.edu,NaN,Office: 32-G638,Computer Science; AI & Decision-making,Massachusetts Institute Technology
74,Virginia Vassilevska Williams,Professor,NaN,NaN,Theory of Computation,NaN,NaN,https://www.eecs.mit.edu/people/virginia-willi...,https://www.eecs.mit.edu/wp-content/uploads/20...,617-253-5851,virgi@mit.edu,NaN,Office: 32-G640,Computer Science; AI & Decision-making,Massachusetts Institute Technology
75,Mengjia Yan,"Homer A. Burnell Career Development Professor,...",NaN,NaN,"Computer Architecture, Security and Cryptography",NaN,NaN,https://www.eecs.mit.edu/people/mengjia-yan/,https://www.eecs.mit.edu/people/mengjia-yan/,617-258-0719,mengjiay@mit.edu,NaN,Office: 32G-840,Computer Science,Massachusetts Institute Technology
76,Nickolai Zeldovich,Professor of EECS,NaN,NaN,Programming Languages and Software Engineering...,NaN,NaN,https://www.eecs.mit.edu/people/nickolai-zeldo...,https://www.eecs.mit.edu/people/nickolai-zeldo...,(617) 253-6005,nickolai@csail.mit.edu,NaN,Office: 32-G994,Computer Science,Massachusetts Institute Technology


In [7]:
names = df["name"].to_list()

In [8]:
len(names), names 

(78,
 ['Hal Abelson',
  'Anant Agarwal',
  'Pulkit Agrawal',
  'Mohammad Alizadeh',
  'Saman Amarasinghe',
  'Arvind',
  'Hari Balakrishnan',
  'Adam Belay',
  'Robert Berwick',
  'Michael Carbin',
  'Anantha Chandrakasan',
  'Brynmor Chapman',
  'Adam Chlipala',
  'Isaac Chuang',
  'Henry Corrigan-Gibbs',
  'Constantinos Daskalakis',
  'Christina Delimitrou',
  'Erik Demaine',
  'Srini Devadas',
  'Frederic Durand',
  'Joel Emer',
  'Mohsen Ghaffari',
  'Manya Ghobadi',
  'David Gifford',
  'Shafrira Goldwasser',
  'W. Eric L Grimson',
  'John Guttag',
  'Song Han',
  'Sam Hopkins',
  'Daniel Huttenlocher',
  'Piotr Indyk',
  'Daniel Jackson',
  'M. Frans Kaashoek',
  'David Karger',
  'Dina Katabi',
  'Kyle Keane',
  'Manolis Kellis',
  'Tim Kraska',
  'Butler Lampson',
  'Steven Leeb',
  'Charles Leiserson',
  'Barbara Liskov',
  'Nancy Lynch',
  'Samuel Madden',
  'Aleksander Mądry',
  'Wojciech Matusik',
  'Muriel Médard',
  'Silvio Micali',
  'Robert Miller',
  'Robert Morris',
 

## fetch google scholar data for all CS faculty

In [9]:
ntest = -1  #  2 # 

In [10]:
pub_data = []
for n, name in enumerate(names[:ntest]):
#     if n < 5: continue 
    print(f"n, name = {n}, {name} ...")
    
    author_org = f"{name} {org2}"
    norm_auth_org = normalize_str(author_org)
    file_author = f"data/GScholar_{norm_auth_org}.json"

    try:
        search_query = scholarly.search_author(author_org)
        init_result = next(search_query)
    except Exception as ex:
        print(str(ex))
        init_result = None 
        
    if init_result is None:
        print(f"Failed search_author()")
        continue

    scholar_id = init_result.get("scholar_id", "")
    if not scholar_id:
        print(f"Missing scholar_id")
        continue        
    
    # fetch data
    author = scholarly.fill(init_result)
    
    author_dict = {}
    # fill data cell
    author_dict["name"] = name 
    author_dict["file_author"] = file_author 
    author_dict["scholar_id"] = scholar_id 
    author_dict["affiliation"] = author.get("affiliation", "")
    author_dict["interests"] = "; ".join(author.get("interests", []))
    author_dict["url_author"] = get_scholar_page(scholar_id)
    author_dict["url_picture"] = author.get("url_picture", "")
    author_dict["url_homepage"] = author.get("homepage", "")
    author_dict["citedby"] = author.get("citedby", 0)
    author_dict["citedby5y"] = author.get("citedby5y", 0)
    author_dict["hindex"] = author.get("hindex", 0)
    author_dict["hindex5y"] = author.get("hindex5y", 0)
    author_dict["i10index"] = author.get("i10index", 0)
    author_dict["i10index5y"] = author.get("i10index5y", 0)
    author_dict["num_papers"] = len(author.get("publications", []))
    author_dict["num_coauthors"] = len(author.get("coauthors", []))
      
    # fill row 
    author_data = []
    for c in SCHOLAR_HEADER:
        author_data.append(author_dict.get(c))
        
    # accumulate row
    pub_data.append(author_data)
    
    # persist author data
    with open(Path(file_author), "w", encoding="utf-8") as f:
        f.write(json.dumps(author))
    
    delay = randint(1,5)
    sleep(delay)

n, name = 0, Hal Abelson ...

Failed search_author()
n, name = 1, Anant Agarwal ...
n, name = 2, Pulkit Agrawal ...
n, name = 3, Mohammad Alizadeh ...
n, name = 4, Saman Amarasinghe ...
n, name = 5, Arvind ...
n, name = 6, Hari Balakrishnan ...
n, name = 7, Adam Belay ...
n, name = 8, Robert Berwick ...
n, name = 9, Michael Carbin ...
n, name = 10, Anantha Chandrakasan ...
n, name = 11, Brynmor Chapman ...

Failed search_author()
n, name = 12, Adam Chlipala ...
n, name = 13, Isaac Chuang ...

Failed search_author()
n, name = 14, Henry Corrigan-Gibbs ...
n, name = 15, Constantinos Daskalakis ...
n, name = 16, Christina Delimitrou ...
n, name = 17, Erik Demaine ...
n, name = 18, Srini Devadas ...
n, name = 19, Frederic Durand ...

Failed search_author()
n, name = 20, Joel Emer ...
n, name = 21, Mohsen Ghaffari ...
n, name = 22, Manya Ghobadi ...
n, name = 23, David Gifford ...

Failed search_author()
n, name = 24, Shafrira Goldwasser ...

Failed search_author()
n, name = 25, W. Eric L Gr

In [11]:
len(pub_data)

58

## write out xlsx

In [12]:
df_out = pd.DataFrame(pub_data, columns=SCHOLAR_HEADER)

In [13]:
df_out.head()

,name,affiliation,interests,num_papers,num_coauthors,citedby,hindex,i10index,citedby5y,hindex5y,i10index5y,scholar_id,url_author,url_picture,url_homepage,file_author
0,Anant Agarwal,"Founder and CEO of edX, Professor of Electrica...",Online learning; computer architecture; comput...,476,0,25859,74,207,4369,32,87,E6XXUFcAAAAJ,https://scholar.google.com/citations?user=E6XX...,https://scholar.google.com/citations?view_op=m...,https://people.csail.mit.edu/agarwal/,data/GScholar_anant_agarwal_mit.json
1,Pulkit Agrawal,Massachusetts Institute of Technology,Robotics; Computer Vision; Artificial Intellig...,97,39,8748,30,44,7923,30,43,UpZmJI0AAAAJ,https://scholar.google.com/citations?user=UpZm...,https://scholar.google.com/citations?view_op=m...,https://people.csail.mit.edu/pulkitag/,data/GScholar_pulkit_agrawal_mit.json
2,Mohammad Alizadeh,"Associate Professor of Computer Science, MIT",Computer networks; Systems; Machine learning,164,180,16853,59,103,13727,54,100,6_cxCKQAAAAJ,https://scholar.google.com/citations?user=6_cx...,https://scholar.google.com/citations?view_op=m...,http://people.csail.mit.edu/alizadeh,data/GScholar_mohammad_alizadeh_mit.json
3,Saman Amarasinghe,MIT,Compilers; Performance Engineering; Programmin...,349,85,27057,74,169,7914,41,100,cF6i_goAAAAJ,https://scholar.google.com/citations?user=cF6i...,https://scholar.google.com/citations?view_op=m...,http://people.csail.mit.edu/saman,data/GScholar_saman_amarasinghe_mit.json
4,Arvind,MIT,Computer Architecture,190,0,7687,46,100,1763,19,28,_BqpjCgAAAAJ,https://scholar.google.com/citations?user=_Bqp...,https://scholar.google.com/citations?view_op=m...,http://csg.csail.mit.edu/Users/arvind/,data/GScholar_arvind_mit.json


In [14]:
# import xlsxwriter
file_xlsx = f"data/cs-faculty-gscholar-{org2}-{n}.xlsx"
writer = pd.ExcelWriter(Path(file_xlsx), engine='xlsxwriter')
df_out.to_excel(writer, sheet_name=org2, index=False)
writer.save()

C:\Users\p2p2l\AppData\Local\Temp\ipykernel_19304\3066051369.py:5: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
